<a href="https://colab.research.google.com/github/TirendazAcademy/Deep-Learning-with-TensorFlow/blob/main/Transfer-Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

# Data Loading

In [2]:
dataset, info=tfds.load("tf_flowers", as_supervised=True,
                        with_info= True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.


In [3]:
info

tfds.core.DatasetInfo(
    name='tf_flowers',
    full_name='tf_flowers/3.0.1',
    description="""
    A large set of images of flowers
    """,
    homepage='https://www.tensorflow.org/tutorials/load_data/images',
    data_path='/root/tensorflow_datasets/tf_flowers/3.0.1.incompleteJ27GQR',
    file_format=tfrecord,
    download_size=218.21 MiB,
    dataset_size=221.83 MiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=5),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=3670, num_shards=2>,
    },
    citation="""@ONLINE {tfflowers,
    author = "The TensorFlow Team",
    title = "Flowers",
    month = "jan",
    year = "2019",
    url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }""",
)

# Data Preprocessing

In [4]:
dataset_size = info.splits["train"].num_examples
dataset_size

3670

In [5]:
class_names = info.features["label"].names 
class_names

['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']

In [6]:
n_classes = info.features["label"].num_classes
n_classes

5

In [7]:
test_set_raw,valid_set_raw,train_set_raw=tfds.load(
    "tf_flowers",
    split = ["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True
)

In [8]:
batch_size = 32
preprocess=tf.keras.Sequential([
    tf.keras.layers.Resizing(height=224, width=224,
                             crop_to_aspect_ratio=True),
    tf.keras.layers.Lambda(
        tf.keras.applications.xception.preprocess_input
    )                         
])

In [9]:
train_set=train_set_raw.map(lambda X,y:(preprocess(X),y))
train_set=train_set.shuffle(1000,seed=42).batch(batch_size).prefetch(1)

valid_set=valid_set_raw.map(lambda X,y:(preprocess(X),y)).batch(batch_size)
test_set=test_set_raw.map(lambda X,y:(preprocess(X),y)).batch(batch_size)

In [10]:
data_augmentation=tf.keras.Sequential([
    tf.keras.layers.RandomFlip(mode="horizontal", seed=42),
    tf.keras.layers.RandomContrast(factor=0.2, seed=42),
    tf.keras.layers.RandomRotation(factor=0.05, seed=42)
])

# Model Building

In [11]:
tf.random.set_seed(42)
base_model=tf.keras.applications.xception.Xception(
    weights="imagenet",
    include_top=False
)

83683744/83683744 [==============================] - 3s 0us/step


In [12]:
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output=tf.keras.layers.Dense(n_classes, activation="softmax")(x)

model = tf.keras.Model(inputs=base_model.input,
                       outputs=output)

In [13]:
for layer in base_model.layers:
  layer.trainable = False

In [14]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.SGD(
        learning_rate=0.1,
        momentum=0.9
    ),
    metrics=["accuracy"]
)

# Model Training

In [15]:
history=model.fit(train_set, 
                  validation_data=valid_set,
                  epochs=3)

Epoch 1/3
86/86 [==============================] - 29s 163ms/step - loss: 0.6941 - accuracy: 0.8100 - val_loss: 0.6568 - val_accuracy: 0.8421
Epoch 2/3
86/86 [==============================] - 14s 158ms/step - loss: 0.3132 - accuracy: 0.9106 - val_loss: 0.5465 - val_accuracy: 0.8639
Epoch 3/3
86/86 [==============================] - 14s 155ms/step - loss: 0.2168 - accuracy: 0.9313 - val_loss: 0.5655 - val_accuracy: 0.8657


In [16]:
for indices in zip(range(33), range(33, 66), range(66, 99), range(99, 132)):
    for idx in indices:
        print(f"{idx:3}: {base_model.layers[idx].name:22}", end="")
    print()

  0: input_1                33: block4_pool            66: block8_sepconv1_act    99: block11_sepconv2_act  
  1: block1_conv1           34: batch_normalization_2  67: block8_sepconv1       100: block11_sepconv2      
  2: block1_conv1_bn        35: add_2                  68: block8_sepconv1_bn    101: block11_sepconv2_bn   
  3: block1_conv1_act       36: block5_sepconv1_act    69: block8_sepconv2_act   102: block11_sepconv3_act  
  4: block1_conv2           37: block5_sepconv1        70: block8_sepconv2       103: block11_sepconv3      
  5: block1_conv2_bn        38: block5_sepconv1_bn     71: block8_sepconv2_bn    104: block11_sepconv3_bn   
  6: block1_conv2_act       39: block5_sepconv2_act    72: block8_sepconv3_act   105: add_9                 
  7: block2_sepconv1        40: block5_sepconv2        73: block8_sepconv3       106: block12_sepconv1_act  
  8: block2_sepconv1_bn     41: block5_sepconv2_bn     74: block8_sepconv3_bn    107: block12_sepconv1      
  9: block2_sepconv

In [17]:
for layer in base_model.layers[56:]:
  layer.trainable = True

In [18]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.SGD(
        learning_rate=0.1,
        momentum=0.9
    ),
    metrics=["accuracy"]
)

In [19]:
history=model.fit(train_set, 
                  validation_data=valid_set,
                  epochs=10)

Epoch 1/10
86/86 [==============================] - 38s 297ms/step - loss: 0.5167 - accuracy: 0.8419 - val_loss: 4.4279 - val_accuracy: 0.5426
Epoch 2/10
86/86 [==============================] - 27s 303ms/step - loss: 0.2623 - accuracy: 0.9182 - val_loss: 0.4342 - val_accuracy: 0.8838
Epoch 3/10
86/86 [==============================] - 28s 302ms/step - loss: 0.1186 - accuracy: 0.9618 - val_loss: 0.5934 - val_accuracy: 0.8403
Epoch 4/10
86/86 [==============================] - 27s 303ms/step - loss: 0.0605 - accuracy: 0.9797 - val_loss: 0.8720 - val_accuracy: 0.8131
Epoch 5/10
86/86 [==============================] - 28s 311ms/step - loss: 0.0524 - accuracy: 0.9836 - val_loss: 0.3349 - val_accuracy: 0.9002
Epoch 6/10
86/86 [==============================] - 26s 296ms/step - loss: 0.0292 - accuracy: 0.9924 - val_loss: 0.3197 - val_accuracy: 0.8984
Epoch 7/10
86/86 [==============================] - 27s 303ms/step - loss: 0.0151 - accuracy: 0.9938 - val_loss: 0.3806 - val_accuracy: 0.9020

##Resource:
- [Hands-on Machine Learning with Scikit-Learn, Keras and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781098125967/)

[YouTube](https://www.youtube.com/c/TirendazAcademy) | [Medium](https://tirendazacademy.medium.com) | [Twitter](https://twitter.com/TirendazAcademy) | [Instagram](https://www.instagram.com/tirendazacademy) | [GitHub](https://github.com/TirendazAcademy) | [LinkedIn](https://www.linkedin.com/in/tirendaz-academy)